In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from datasetB_main import MyDataset
import numpy as np
import matplotlib.pyplot as plt
import tempfile
import os
import datetime
import platform

print('Python version:', platform.python_version())
print('Tensorflow version:', tf.__version__)
print('Keras version:', tf.keras.__version__)

In [ ]:
import shutil
if os.path.exists('./tmp'):
	shutil.rmtree('./tmp')
tmp_dir = tempfile.mkdtemp()

builder = MyDataset(data_dir='tmp')
builder.download_and_prepare(
	download_dir='tmp',
	download_config=tfds.download.DownloadConfig(manual_dir='tmp')
)
dataset_train_raw = builder.as_dataset(split='train')
dataset_test_raw = builder.as_dataset(split='test')
dataset_val_raw = builder.as_dataset(split='val')
dataset_info = builder.info

print(dataset_info)

In [ ]:
print('Raw train dataset:', dataset_train_raw)
print('Raw train dataset size:', len(dataset_train_raw), '\n')

print('Raw test dataset:', dataset_test_raw)
print('Raw test dataset size:', len(dataset_test_raw), '\n')

print('Raw val dataset:', dataset_val_raw)
print('Raw val dataset size:', len(dataset_val_raw), '\n')

In [ ]:
NUM_TRAIN_EXAMPLES = dataset_info.splits['train'].num_examples
NUM_TEST_EXAMPLES = dataset_info.splits['test'].num_examples
NUM_VAL_EXAMPLES = dataset_info.splits['val'].num_examples
NUM_CLASSES = dataset_info.features['label'].num_classes

print('Number of TRAIN examples:', NUM_TRAIN_EXAMPLES)
print('Number of TEST examples:', NUM_TEST_EXAMPLES)
print('Number of VAL examples:', NUM_VAL_EXAMPLES)
print('Number of label classes:', NUM_CLASSES)
label_names = dataset_info.features['label'].names
print(f"Labels (classes) : {label_names}")

In [ ]:
INPUT_IMG_SIZE_ORIGINAL = dataset_info.features['image'].shape[0]
INPUT_IMG_SHAPE_ORIGINAL = dataset_info.features['image'].shape

INPUT_IMG_SIZE_REDUCED = INPUT_IMG_SIZE_ORIGINAL // 2
INPUT_IMG_SHAPE_REDUCED = (
    INPUT_IMG_SIZE_REDUCED,
    INPUT_IMG_SIZE_REDUCED,
    INPUT_IMG_SHAPE_ORIGINAL[2]
)

# Here we may switch between bigger or smaller image sized that we will train our model on.
INPUT_IMG_SIZE = INPUT_IMG_SIZE_REDUCED
INPUT_IMG_SHAPE = INPUT_IMG_SHAPE_REDUCED

print('Input image size (original):', INPUT_IMG_SIZE_ORIGINAL)
print('Input image shape (original):', INPUT_IMG_SHAPE_ORIGINAL)
print('\n')
print('Input image size (reduced):', INPUT_IMG_SIZE_REDUCED)
print('Input image shape (reduced):', INPUT_IMG_SHAPE_REDUCED)
print('\n')
print('Input image size:', INPUT_IMG_SIZE)
print('Input image shape:', INPUT_IMG_SHAPE)

In [ ]:
# Function to convert label ID to labels string.
get_label_name = dataset_info.features['label'].int2str

In [ ]:
print(get_label_name(0))
print(get_label_name(1))
print(get_label_name(2))

In [ ]:
def preview_dataset(dataset, size=12):
    plt.figure(figsize=(size, size))
    plot_index = 0
    nb_rocks = 0
    nb_scissors = 0
    nb_paper = 0
    for image, label in dataset.take(size):
        plot_index += 1
        if get_label_name(label.numpy()) == 'rock':
            nb_rocks += 1
        elif get_label_name(label.numpy()) == 'scissors':
            nb_scissors += 1
        elif get_label_name(label.numpy()) == 'paper':
            nb_paper += 1
        plt.subplot(size//3, size//3, plot_index)
        # plt.axis('Off')
        plt.title('Label: %s' % get_label_name(label.numpy()))
        plt.imshow(image.numpy())
    print(f'rocks: {nb_rocks}, scissors: {nb_scissors}, paper: {nb_paper}')

In [ ]:
# Explore raw training dataset images.
#preview_dataset(dataset_train_raw)

In [ ]:
def format_example(features):
    image = features['image']
    label = features['label']
    # Make image color values to be float.
    image = tf.cast(image, tf.float32)
    # Make image color values to be in [0..1] range.
    image = image / 255.
    # Make sure that image has a right size
    image = tf.image.resize(image, [INPUT_IMG_SIZE, INPUT_IMG_SIZE])
    return image, label

In [ ]:
print(dataset_train_raw)
dataset_train = dataset_train_raw.map(format_example)
dataset_test = dataset_test_raw.map(format_example)
dataset_val = dataset_val_raw.map(format_example)

In [ ]:
BATCH_SIZE = 32

dataset_train = dataset_train.batch(BATCH_SIZE)

DATASET_TEST = dataset_test.batch(BATCH_SIZE)

DATASET_VAL = dataset_val.batch(BATCH_SIZE)

In [ ]:
#Load model
model = tf.keras.models.load_model('rock_paper_scissors_cnn.keras')

model.summary()

In [ ]:
# Test model with our dataset without fit
model.evaluate(DATASET_TEST)

# Active Learning part

In [ ]:
budgets = [0.01, 0.02, 0.05, 0.1, 0.2, 0.3]

In [ ]:
# Preparing callbacks.
os.makedirs('logs/fit', exist_ok=True)
tensorboard_log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d-%H%M%S')
tensorboard_callback = tf.keras.callbacks.TensorBoard(
    log_dir=tensorboard_log_dir,
    histogram_freq=1
)

os.makedirs('tmp/checkpoints', exist_ok=True)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath='tmp/checkpoints/weights.{epoch:02d}-{val_loss:.2f}.keras'
)

early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=5,
    monitor='val_accuracy'
    # monitor='val_loss'
)

In [ ]:
import random

def selectRandomSamples(dataset_train, num_samples):

    images = []
    labels = []

    for image_batch, label_batch in dataset_train:
        images.extend(image_batch.numpy())  
        labels.extend(label_batch.numpy())

    random_indices = [random.randint(0, NUM_TRAIN_EXAMPLES - 1) for _ in range(num_samples)]

    selected_images = [images[idx] for idx in random_indices]
    selected_labels = [labels[idx] for idx in random_indices]

    dataset_train_random = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_random = dataset_train_random.shuffle(buffer_size=num_samples)
    dataset_train_random = dataset_train_random.batch(BATCH_SIZE)

    return dataset_train_random

In [ ]:
def least_confidence(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons les moins confiants pour l'entraînement basé sur un modèle de deep learning.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement), 
                           représentant le pourcentage de données les moins confiantes à utiliser.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (les moins confiants).
    """
    # Liste pour stocker les incertitudes et les indices
    uncertainties = []
    images = []
    labels = []

    # Parcourir le dataset en batch
    for image_batch, label_batch in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        batch_uncertainties = 1 - np.max(preds.numpy(), axis=1)  # Calculer l'incertitude pour chaque image
        uncertainties.extend(batch_uncertainties)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Trier les incertitudes et récupérer les indices des plus incertains
    sorted_indices = np.argsort(uncertainties)[:num_samples]

    # Sélectionner les images et labels correspondants aux indices les moins confiants
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_least_confidence = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_least_confidence = dataset_train_least_confidence.shuffle(buffer_size=num_samples)
    dataset_train_least_confidence = dataset_train_least_confidence.batch(BATCH_SIZE)

    return dataset_train_least_confidence


In [ ]:
from scipy.stats import entropy

def entropy_based_sampling(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons avec la plus grande entropie pour l'entraînement basé sur un modèle de deep learning.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement), 
                           représentant les données les plus incertaines à utiliser.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (avec la plus grande entropie).
    """
    # Liste pour stocker les entropies et les indices
    entropies = []
    images = []
    labels = []

    # Parcourir le dataset en batch
    for image_batch, label_batch in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        
        # Appliquer la fonction d'entropie de SciPy pour chaque batch
        batch_entropies = entropy(preds.numpy().T)  # Entropie de Shannon pour chaque image
        entropies.extend(batch_entropies)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Trier les entropies et récupérer les indices des plus incertains (avec la plus grande entropie)
    sorted_indices = np.argsort(entropies)[-num_samples:]

    # Sélectionner les images et labels correspondants aux indices les plus incertains
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_entropy_based = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_entropy_based = dataset_train_entropy_based.shuffle(buffer_size=num_samples)
    dataset_train_entropy_based = dataset_train_entropy_based.batch(BATCH_SIZE)

    return dataset_train_entropy_based

In [ ]:
def margin_based_sampling(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons avec la plus petite marge pour l'entraînement basé sur un modèle de deep learning.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement),
                           représentant les données avec la plus petite marge à utiliser.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (avec la plus petite marge).
    """
    # Liste pour stocker les marges et les indices
    margins = []
    images = []
    labels = []

    # Parcourir le dataset en batch
    for image_batch, label_batch in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        
        # Calculer la marge : différence entre les deux plus grandes probabilités
        top2_probs = np.partition(preds.numpy(), -2)[:, -2:]  # Top 2 plus grandes valeurs de probabilité
        batch_margins = top2_probs[:, 1] - top2_probs[:, 0]  # Calcul de la marge (différence entre les deux plus grandes)
        
        margins.extend(batch_margins)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Trier les marges et récupérer les indices des plus petites marges
    sorted_indices = np.argsort(margins)[:num_samples]

    # Sélectionner les images et labels correspondants aux indices des plus petites marges
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_margin_based = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    preview_dataset(dataset_train_margin_based, num_samples)
    dataset_train_margin_based = dataset_train_margin_based.shuffle(buffer_size=num_samples)
    dataset_train_margin_based = dataset_train_margin_based.batch(BATCH_SIZE)

    return dataset_train_margin_based

In [ ]:
def ratio_sampling(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons avec le plus faible ratio entre la probabilité de la classe la plus élevée
    et la somme des autres probabilités, pour l'entraînement basé sur un modèle de deep learning.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement),
                           représentant les données avec le ratio le plus faible à utiliser.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (avec le plus faible ratio).
    """
    # Liste pour stocker les ratios et les indices
    ratios = []
    images = []
    labels = []

    # Parcourir le dataset en batch
    for image_batch, label_batch in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        
        # Calculer le ratio : probabilité de la classe la plus probable / somme des autres probabilités
        top_prob = np.max(preds.numpy(), axis=1)  # Probabilité de la classe la plus probable
        sum_probs = np.sum(preds.numpy(), axis=1) - top_prob  # Somme des autres probabilités
        batch_ratios = top_prob / (sum_probs + 1e-10)  # Calcul du ratio (ajout d'un petit epsilon pour éviter la division par zéro)
        
        ratios.extend(batch_ratios)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Trier les ratios et récupérer les indices des plus faibles ratios
    sorted_indices = np.argsort(ratios)[:num_samples]

    # Sélectionner les images et labels correspondants aux indices des plus faibles ratios
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_ratio_based = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_ratio_based = dataset_train_ratio_based.shuffle(buffer_size=num_samples)
    dataset_train_ratio_based = dataset_train_ratio_based.batch(BATCH_SIZE)

    return dataset_train_ratio_based

In [ ]:
from numpy.linalg import norm as euclidean_dis

def model_based_outlier_sampling(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons les plus éloignés des prédictions moyennes du modèle (outliers).
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement),
                           représentant les outliers prédits par le modèle.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (outliers).
    """
    # Liste pour stocker les distances aux prédictions moyennes
    distances = []
    images = []
    labels = []

    # Parcourir le dataset en batch pour collecter toutes les prédictions
    all_preds = []
    for image_batch, _ in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        all_preds.append(preds.numpy())
    all_preds = np.vstack(all_preds)  # Convertir en une seule matrice
    mean_pred = np.mean(all_preds, axis=0)  # Moyenne des prédictions globales

    # Calculer la distance entre chaque prédiction et la prédiction moyenne
    for image_batch, label_batch in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        batch_distances = euclidean_dis(preds.numpy() - mean_pred, axis=1)  # Distance euclidienne
        distances.extend(batch_distances)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Trier les distances et récupérer les indices des outliers (distances les plus grandes)
    sorted_indices = np.argsort(distances)[-num_samples:]

    # Sélectionner les images et labels correspondants aux indices des outliers
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_outliers = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_outliers = dataset_train_outliers.shuffle(buffer_size=num_samples)
    dataset_train_outliers = dataset_train_outliers.batch(BATCH_SIZE)

    return dataset_train_outliers


In [ ]:
from sklearn.cluster import KMeans

def cluster_based_sampling(model, dataset_train, num_samples, num_clusters=10):
    """
    Sélectionne les échantillons basés sur le clustering pour l'entraînement actif.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow préalablement entraîné pour extraire des représentations.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement).
        num_clusters (int): Nombre de clusters à former dans l'espace des représentations.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés.
    """
    # Listes pour stocker les représentations, les images et les labels
    embeddings = []
    images = []
    labels = []

    # Parcourir le dataset pour extraire les représentations
    for image_batch, label_batch in dataset_train:
        batch_embeddings = model(image_batch, training=False).numpy()  # Extraire les représentations
        embeddings.extend(batch_embeddings)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    embeddings = np.array(embeddings)

    # Appliquer l'algorithme de clustering KMeans
    kmeans = KMeans(n_clusters=num_clusters)
    cluster_labels = kmeans.fit_predict(embeddings)

    # Sélectionner les échantillons les plus incertains ou les plus représentatifs de chaque cluster
    selected_indices = []
    for cluster in range(num_clusters):
        # Trouver les indices des points appartenant au cluster
        cluster_indices = np.where(cluster_labels == cluster)[0]

        # Si le cluster contient des points, choisir un échantillon central
        if len(cluster_indices) > 0:
            # Calculer les distances par rapport au centroïde
            cluster_center = kmeans.cluster_centers_[cluster]
            distances = np.linalg.norm(embeddings[cluster_indices] - cluster_center, axis=1)

            # Sélectionner l'index du point le plus proche du centroïde (point incertain)
            selected_index = cluster_indices[np.argmin(distances)]
            selected_indices.append(selected_index)

    # Sélectionner les images et labels correspondants
    selected_images = [images[idx] for idx in selected_indices]
    selected_labels = [labels[idx] for idx in selected_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_cluster_based = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_cluster_based = dataset_train_cluster_based.shuffle(buffer_size=num_samples)
    dataset_train_cluster_based = dataset_train_cluster_based.batch(BATCH_SIZE)  # Définir la taille de batch

    return dataset_train_cluster_based

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

def representative_sampling(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons les plus représentatifs pour l'entraînement basé sur un modèle de deep learning.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour extraire des représentations.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement), 
                           représentant les données les plus représentatives.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (les plus représentatifs).
    """
    # Listes pour stocker les représentations, les images et les labels
    embeddings = []
    images = []
    labels = []

    # Parcourir le dataset pour extraire les représentations
    for image_batch, label_batch in dataset_train:
        batch_embeddings = model(image_batch, training=False).numpy()  # Extraire les représentations
        embeddings.extend(batch_embeddings)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Calculer la distance moyenne de chaque point à tous les autres
    distance_matrix = euclidean_distances(embeddings, embeddings)
    mean_distances = np.mean(distance_matrix, axis=1)

    # Trier les distances moyennes et récupérer les indices des plus petites (les plus représentatives)
    sorted_indices = np.argsort(mean_distances)[:num_samples]

    # Sélectionner les images et labels correspondants
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_representative = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    preview_dataset(dataset_train_representative, num_samples)
    dataset_train_representative = dataset_train_representative.shuffle(buffer_size=num_samples)
    dataset_train_representative = dataset_train_representative.batch(BATCH_SIZE)

    return dataset_train_representative

In [ ]:
def combined_sampling_sequential(model, dataset_train, num_samples, rep_samples_ratio=0.5):
    """
    Combine Representative Sampling et Margin of Confidence via une combinaison séquentielle.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre total d'échantillons à sélectionner.
        rep_samples_ratio (float): Proportion d'échantillons pour Representative Sampling avant d'appliquer Margin of Confidence.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons combinés.
    """
    # Étape 1 : Calcul du nombre d'échantillons pour Representative Sampling
    rep_samples = int(NUM_TRAIN_EXAMPLES * rep_samples_ratio)
    print("Échantillons pour Representative Sampling:", rep_samples)

    # Étape 2 : Appliquer Representative Sampling
    print("Étape 1: Representative Sampling")
    rep_dataset = representative_sampling(model, dataset_train, rep_samples)

    # Étape 3 : Appliquer Margin of Confidence sur le sous-ensemble réduit
    print("Étape 2: Margin of Confidence")
    final_dataset = margin_based_sampling(model, rep_dataset, num_samples)

    return final_dataset

In [ ]:
def margin_based_sampling_inversed(model, dataset_train, num_samples):
    """
    Sélectionne les échantillons avec la plus grande marge pour l'entraînement basé sur un modèle de deep learning.
    
    Args:
        model (tf.keras.Model): Modèle TensorFlow entraîné pour effectuer des prédictions.
        dataset_train (tf.data.Dataset): Dataset TensorFlow contenant les données d'entraînement (images, labels).
        num_samples (int): Nombre d'échantillons à sélectionner (parmi l'ensemble d'entraînement),
                           représentant les données avec la plus petite marge à utiliser.

    Returns:
        tf.data.Dataset: Un nouveau dataset TensorFlow contenant les échantillons sélectionnés (avec la plus petite marge).
    """
    # Liste pour stocker les marges et les indices
    margins = []
    images = []
    labels = []

    # Parcourir le dataset en batch
    for image_batch, label_batch in dataset_train:
        preds = model(image_batch, training=False)  # Prédictions sur le batch
        
        # Calculer la marge : différence entre les deux plus grandes probabilités
        top2_probs = np.partition(preds.numpy(), -2)[:, -2:]  # Top 2 plus grandes valeurs de probabilité
        batch_margins = top2_probs[:, 1] - top2_probs[:, 0]  # Calcul de la marge (différence entre les deux plus grandes)
        
        margins.extend(batch_margins)
        images.extend(image_batch.numpy())  # Ajouter les images du batch
        labels.extend(label_batch.numpy())  # Ajouter les labels du batch

    # Trier les marges et récupérer les indices des plus grandes marges
    sorted_indices = np.argsort(margins)[-num_samples:]

    # Sélectionner les images et labels correspondants aux indices des plus grandes marges
    selected_images = [images[idx] for idx in sorted_indices]
    selected_labels = [labels[idx] for idx in sorted_indices]

    # Créer un nouveau dataset TensorFlow à partir des images et labels sélectionnés
    dataset_train_margin_based = tf.data.Dataset.from_tensor_slices((selected_images, selected_labels))
    dataset_train_margin_based = dataset_train_margin_based.shuffle(buffer_size=num_samples)
    dataset_train_margin_based = dataset_train_margin_based.batch(BATCH_SIZE)

    return dataset_train_margin_based

In [ ]:
def retrain(dataset_train, method, budget=0.0):
    model = tf.keras.models.load_model('rock_paper_scissors_cnn.keras')
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    patience=1,
    monitor='val_accuracy'                                                                                              
    )

    num_samples = int(budget * NUM_TRAIN_EXAMPLES)
    print(f'Number of training examples to select: {num_samples}')
    
    if method == 'RANDOM':
        dataset_to_train = selectRandomSamples(dataset_train, num_samples)
    elif method == 'LEAST CONFIDENCE':
        dataset_to_train = least_confidence(model, dataset_train, num_samples)
    elif method == 'ENTROPY BASED':
        dataset_to_train = entropy_based_sampling(model, dataset_train, num_samples)
    elif method == 'RATIO SAMPLING':
        dataset_to_train = ratio_sampling(model, dataset_train, num_samples)
    elif method == 'MARGIN OF CONFIDENCE':
        dataset_to_train = margin_based_sampling(model, dataset_train, num_samples)
    elif method == 'MODEL BASED OUTLIER':
        dataset_to_train = model_based_outlier_sampling(model, dataset_train, num_samples)
    elif method == 'CLUSTER BASED':
        dataset_to_train = cluster_based_sampling(model, dataset_train, num_samples)
    elif method == 'REPRESENTATIVE SAMPLING':
        dataset_to_train = representative_sampling(model, dataset_train, num_samples)
    elif method == 'COMBINED SEQUENTIAL':
        dataset_to_train = combined_sampling_sequential(model, dataset_train, num_samples)
    elif method == 'MARGIN OF CONFIDENCE INVERSED':
        dataset_to_train = margin_based_sampling_inversed(model, dataset_train, num_samples)
        
    model.fit(x=dataset_to_train.repeat(),
    validation_data=DATASET_VAL.repeat(), epochs=50,
    steps_per_epoch=num_samples // BATCH_SIZE,
    validation_steps=NUM_TEST_EXAMPLES // BATCH_SIZE,
    callbacks=[tensorboard_callback, early_stopping_callback], verbose=0)
    
    test_loss, test_accuracy = model.evaluate(DATASET_TEST, verbose=0)
    print(f"Test Accuracy: {test_accuracy}, Test Loss: {test_loss}")
    return test_accuracy

In [ ]:
def getAccuracyStats(dataset_train, method, budget=0.0, repeats=1):
    accuracies = []
    for i in range(repeats):
        accuracies.append(retrain(dataset_train, method, budget))
    average_accuracy = np.mean(accuracies)
    if method == 'RANDOM':
        accuracies = np.array(accuracies)
        std_dev = np.std(accuracies)
        confidence_interval = 1.96 * (std_dev / np.sqrt(repeats))
        min_confidence = average_accuracy - confidence_interval
        max_confidence = average_accuracy + confidence_interval
        return average_accuracy, min_confidence, max_confidence
    return average_accuracy

## Check reproducibility

In [ ]:
for image, label in dataset_train.take(1):  # Prenez le premier batch
    first_image = image[0].numpy()
    first_label = label[0].numpy()
    break

In [ ]:
for image, label in dataset_train.take(1):  # Prenez le premier batch
    second_image = image[0].numpy()
    second_label = label[0].numpy()
    break

In [ ]:
assert np.array_equal(second_image, first_image)
assert second_label == first_label

## RANDOM METHOD

### Random sampling method

In [ ]:
random_averages_accuracy = []
random_min_confidence = []
random_max_confidence = []
nb_repeats = 5

for budget in budgets:
    average_accuracy, min_confidence, max_confidence = getAccuracyStats(dataset_train, 'RANDOM', budget, nb_repeats)
    random_averages_accuracy.append(average_accuracy)
    random_min_confidence.append(min_confidence)
    random_max_confidence.append(max_confidence)

## UNCERTAINTY METHODS

### Method n°1 : LEAST CONFIDENCE METHOD

In [ ]:
least_confidence_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'LEAST CONFIDENCE', budget)
    least_confidence_accuracy.append(accuracy)

### Method n°2 : ENTROPY BASED METHOD

In [ ]:
entropy_based_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'ENTROPY BASED', budget)
    entropy_based_accuracy.append(accuracy)

### Method n°3 : MARGIN OF CONFIDENCE METHOD

In [ ]:
margin_of_confidence_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'MARGIN OF CONFIDENCE', budget)
    margin_of_confidence_accuracy.append(accuracy)

### Method n°4 : Ratio Sampling Method

In [ ]:
ratio_sampling_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'RATIO SAMPLING', budget)
    ratio_sampling_accuracy.append(accuracy)

## DIVERSITY METHODS

### Method n°1 : Model based outlier sampling

In [ ]:
model_based_outlier_sampling_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'MODEL BASED OUTLIER', budget)
    model_based_outlier_sampling_accuracy.append(accuracy)

### Method n°2 : Cluster based sampling

In [ ]:
cluster_based_sampling_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'CLUSTER BASED', budget)
    cluster_based_sampling_accuracy.append(accuracy)

### Method n°3 : Representative sampling

In [ ]:
representative_sampling_accuracy = []
#for budget in budgets:
accuracy = getAccuracyStats(dataset_train, 'REPRESENTATIVE SAMPLING', 0.05)
representative_sampling_accuracy.append(accuracy)

## Advanced Learning (Combined method)

### Combined method n°1 : Representative sampling + Ratio sampling methods

In [ ]:
combined_methods_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'COMBINED SEQUENTIAL', budget)
    combined_methods_accuracy.append(accuracy)

## GRAPHS

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(budgets, least_confidence_accuracy, label='Least confidence accuracy')
plt.plot(budgets, entropy_based_accuracy, label='Entropy based accuracy')
plt.plot(budgets, margin_of_confidence_accuracy, label='Margin of confidence accuracy')
plt.plot(budgets, ratio_sampling_accuracy, label='Ratio sampling accuracy')
plt.plot(budgets, model_based_outlier_sampling_accuracy, label='Model based outlier sampling accuracy')
plt.plot(budgets, cluster_based_sampling_accuracy, label='Cluster based accuracy')
plt.plot(budgets, representative_sampling_accuracy, label='Representative sampling accuracy')
plt.plot(budgets, combined_methods_accuracy, label='Advanced active learning accuracy')
plt.plot(budgets, random_averages_accuracy, label='Random average accuracy')
plt.vlines(budgets, random_min_confidence, random_max_confidence, color='gray', label='Intervalle de confiance à 95%')
plt.xlabel('budget of training data')
plt.ylabel('Accuracy')
plt.title('Accuracy for different budgets of training data with different active learning methods')
plt.legend()
plt.grid(linestyle='--', linewidth=1, alpha=0.5)
plt.savefig('active_learning_accuracy_different_active_learning_methods_.png')
plt.show()

In [ ]:
margin_of_confidence_inversed_accuracy = []
for budget in budgets:
    accuracy = getAccuracyStats(dataset_train, 'MARGIN OF CONFIDENCE INVERSED', budget)
    margin_of_confidence_inversed_accuracy.append(accuracy)

In [ ]:
plt.figure(figsize=(12, 6))
plt.plot(budgets, margin_of_confidence_accuracy, label='Margin of confidence accuracy')
plt.plot(budgets, margin_of_confidence_inversed_accuracy, label='Margin of confidence inversed accuracy')
plt.plot(budgets, random_averages_accuracy, label='Random average accuracy')
plt.vlines(budgets, random_min_confidence, random_max_confidence, color='gray', label='Intervalle de confiance à 95%')
plt.xlabel('budget of training data')
plt.ylabel('Accuracy')
plt.title('Accuracy for different budgets of training data with different active learning methods')
plt.legend()
plt.grid(linestyle='--', linewidth=1, alpha=0.5)
plt.savefig('active_learning_accuracy_different_active_learning_methods_after_hypothesis.png')
plt.show()